<a href="https://colab.research.google.com/github/marco-siino/GM_SOURCE_CODE/blob/main/NVIDIA_DS/GM_Fine_Tuning_Code_T5_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Setup Development Environment

Our first step is to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

In [ ]:
# !pip install pytesseract transformers datasets evaluate rouge-score nltk tensorboard py7zr --upgrade
!pip install pytesseract transformers==4.28.1 datasets evaluate rouge-score nltk tensorboard py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.6 MB

This example will use the [Hugging Face Hub](https://huggingface.co/models) as a remote model versioning service. To be able to push our model to the Hub, you need to register on the [Hugging Face](https://huggingface.co/join).
If you already have an account, you can skip this step.
After you have an account, we will use the `notebook_login` util from the `huggingface_hub` package to log into our account and store our token (access key) on the disk.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Import dependencies

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets
from sklearn.model_selection import KFold

## 2. Load and prepare imdb dataset

we will use the [imdb](https://huggingface.co/datasets/imdb) Large Movie Review Dataset. This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.


In [ ]:
dataset_id = "imdb"

To load the `samsum` dataset, we use the `load_dataset()` method from the 🤗 Datasets library.


In [ ]:
# Import the csv into pandas dataframe and add the headers

df = pd.read_csv('dataset-devmap-nvidia.csv',sep=',')
#df = pd.concat(map(pd.read_csv, ['dataset-devmap-nvidia.csv', 'dataset-devmap-amd.csv']))
#df = pd.read_csv('dataset-devmap-nvidia.csv', sep='\t', names=['benchmark','dataset','comp','rational','mem','localmem','coalesced','atomic','transfer','wgsize','oracle','runtime_cpu','runtime_gpu','src','seq'])
#print(df.head())
# Now include transfer and wgsize columns into the src column.
df['src'] = df['transfer'].astype(str) +" - "+ df['wgsize'].astype(str) +" - "+df["src"]
# # Removing unwanted columns and only leaving title of news and the category which will be the target
df = df[['src','oracle']]
#print(df.head())

encode_dict = {}

def encode_cat(x):
    if x == "GPU":
      encode_dict[x]=1
    else:
      encode_dict[x]=0
    return encode_dict[x]

df['ENCODE_CAT'] = df['oracle'].apply(lambda x: encode_cat(x))

print(df)

df = df.sample(frac=1, random_state=1).reset_index(drop=True)

print(df)

                                                   src oracle  ENCODE_CAT
0    2048 - 255 - __kernel void A(int a, const __gl...    GPU           1
1    131072 - 256 - __kernel void A(__global uint* ...    GPU           1
2    3145728 - 256 - extern void B(float4 a, float4...    GPU           1
3    4096 - 256 - __kernel void A(__global float* a...    GPU           1
4    524288 - 256 - __kernel void A(__global uint* ...    CPU           0
..                                                 ...    ...         ...
675  2000628 - 128 - __kernel void A(__global const...    CPU           0
676  2000628 - 128 - __kernel void A(__global const...    CPU           0
677  71647488 - 0 - extern int D(__private int, __p...    CPU           0
678  71647488 - 256 - extern int B(int, int, int);\...    CPU           0
679  117440512 - 128 - __kernel void A(__global con...    CPU           0

[680 rows x 3 columns]
                                                   src oracle  ENCODE_CAT
0    6346800 -

In [ ]:
df = df[['src','ENCODE_CAT']]
df= df.rename(columns={"src": "text", "ENCODE_CAT": "label"})
print(df)

                                                  text  label
0    6346800 - 52 - __kernel void A(__global double...      0
1    15949464 - 48 - __kernel void A(__global const...      1
2    14742140 - 128 - extern double __clc_pow(const...      0
3    15948384 - 2 - extern void B(double m, double ...      0
4    755867256 - 64 - __kernel void A(__global doub...      1
..                                                 ...    ...
675  14742140 - 128 - __kernel void A(__global doub...      1
676  57594120 - 128 - __kernel void A(__global cons...      0
677  320454824 - 64 - extern void B(double, double,...      1
678  1344811536 - 26 - extern void D(__private doub...      1
679  1313673512 - 32 - __kernel void A(__global con...      1

[680 rows x 2 columns]


In [ ]:
# Each i-train fold can be accessed with df_train[i]. Same for test.
fold_nr = 5

kf = KFold(n_splits=5, random_state=None, shuffle=False)

df_train = []
df_test = []
model = []

for i, (train_index, test_index) in enumerate(kf.split(df)):
  df_train.append(df.iloc[train_index])
  df_test.append(df.iloc[test_index])
# print(df_train[0])

for i in range(0,fold_nr):
  df_train[i] = df_train[i].reset_index(drop=True)
  df_test[i] = df_test[i].reset_index(drop=True)

# Old on IMDB dataset.

In [ ]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 25000
# Test dataset size: 25000

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Train dataset size: 25000
Test dataset size: 25000


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Lets checkout an example of the dataset.

In [ ]:
dataset['train'][1]

{'text': '"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, 

# To train our model we need to convert our inputs (text) to token IDs. This is done by a 🤗 Transformers Tokenizer. If you are not sure what this means check out [chapter 6](https://huggingface.co/course/chapter6/1?fw=tf) of the Hugging Face Course.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="Salesforce/codet5-small"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

before we can start training we need to preprocess our data. Abstractive flan t5 is good for a text2text-generation task. This means our model will take a text as input and generate a text as output. For this we want to understand how long our input and output will be to be able to efficiently batch our data.
- as result, we should to convert label from int to string

In [ ]:
import pandas as pd
from datasets import Dataset
import random

# dataset['train'] = dataset['train'].shuffle(seed=42).select(range(2000))
# dataset['test'] = dataset['test'].shuffle(seed=42).select(range(1000))

#dataset['train'] = dataset['train'].shuffle(seed=42)

# per ora faccio solo fold 0.
#train_df = pd.DataFrame(dataset['train'])
#test_df = pd.DataFrame(dataset['test'])
train_df = df_train[0]
test_df = df_test[0]


dataset.clear()
train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 544
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 136
    })
})

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Max target length: 3


In [ ]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/544 [00:00<?, ? examples/s]

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate FLAN-T5

After we have processed our dataset, we can start training our model. Therefore we first need to load our [FLAN-T5](https://huggingface.co/models?search=flan-t5) from the Hugging Face Hub. In the example we are using a instance with a NVIDIA V100 meaning that we will fine-tune the `base` version of the model.
_I plan to do a follow-up post on how to fine-tune the `xxl` version of the model using Deepspeed._


In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="Salesforce/codet5-small"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

We want to evaluate our model during training. The `Trainer` supports evaluation during training by providing a `compute_metrics`.  
The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries

We are going to use `evaluate` library to evaluate the `rogue` score.

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
#metric = evaluate.load("f1")
metric = evaluate.load("accuracy")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    #result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Before we can start training is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training. We are leveraging the [Hugging Face Hub](https://huggingface.co/models) integration of the `Trainer` to automatically push our checkpoints, logs and metrics during training into a repository.

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-imdb-text-classification"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-4,

    num_train_epochs=10,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="epoch",
    # logging_steps=1000,
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

We can start our training by using the `train` method of the `Trainer`.

In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
544,0.328800
1088,0.247700
1632,0.222300
2176,0.205100
2720,0.181000
3264,0.201000
3808,0.162000
4352,0.143900
4896,0.122200
5440,0.098100


TrainOutput(global_step=5440, training_loss=0.19120708423502306, metrics={'train_runtime': 447.0723, 'train_samples_per_second': 12.168, 'train_steps_per_second': 12.168, 'total_flos': 736259400007680.0, 'train_loss': 0.19120708423502306, 'epoch': 10.0})

Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


In [ ]:
trainer.evaluate()

{'eval_loss': 0.15480194985866547,
 'eval_accuracy': 86.0294,
 'eval_gen_len': 3.0,
 'eval_runtime': 8.3504,
 'eval_samples_per_second': 16.287,
 'eval_steps_per_second': 16.287,
 'epoch': 10.0}

In [ ]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

## 4. Run Inference and Classification Report

In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = dataset['test']['text'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['label'][i])

  progress_bar.update(1)

  0%|          | 0/136 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.61 GiB. GPU 0 has a total capacty of 14.75 GiB of which 2.65 GiB is free. Process 3242 has 12.09 GiB memory in use. Of the allocated memory 8.46 GiB is allocated by PyTorch, and 3.50 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
str_labels_list = []
for i in range(len(labels_list)): str_labels_list.append(str(labels_list[i]))

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.82      0.77        66
           1       0.80      0.70      0.75        70

    accuracy                           0.76       136
   macro avg       0.76      0.76      0.76       136
weighted avg       0.76      0.76      0.76       136



In [ ]:
str_labels_list[0]

'0'

In [ ]:
predictions_list

[]

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)